In [2]:
import tensorflow as tf 
import numpy as np
import pandas as pd
import datetime, os
%load_ext tensorboard


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [24]:
# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)

In [3]:
data = pd.read_csv("/Users/xxx/Documents/Jedha/final_project/Data/HAM10000_metadata.csv")
data.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear


In [4]:
data.shape

(10015, 7)

In [5]:
data["image_id"] = data["image_id"].astype("str") + ".jpg"
data.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization
0,HAM_0000118,ISIC_0027419.jpg,bkl,histo,80.0,male,scalp
1,HAM_0000118,ISIC_0025030.jpg,bkl,histo,80.0,male,scalp
2,HAM_0002730,ISIC_0026769.jpg,bkl,histo,80.0,male,scalp
3,HAM_0002730,ISIC_0025661.jpg,bkl,histo,80.0,male,scalp
4,HAM_0001466,ISIC_0031633.jpg,bkl,histo,75.0,male,ear


In [6]:
#Rows suffle (because the values in the target column "dx" are grouped by value, it creates a bias)
data = data.sample(frac=1,random_state=0).reset_index(drop=True)

In [7]:
data.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization
0,HAM_0002695,ISIC_0028664.jpg,nv,follow_up,45.0,male,back
1,HAM_0000370,ISIC_0025998.jpg,nv,follow_up,70.0,male,trunk
2,HAM_0006372,ISIC_0032817.jpg,nv,histo,35.0,male,back
3,HAM_0006835,ISIC_0026577.jpg,nv,histo,75.0,male,chest
4,HAM_0005536,ISIC_0026798.jpg,bcc,histo,45.0,male,lower extremity


In [8]:
data.shape

(10015, 7)

In [9]:
#Check of missing values
print(100*data.isna().sum()/data.shape[0])

lesion_id       0.000000
image_id        0.000000
dx              0.000000
dx_type         0.000000
age             0.569146
sex             0.000000
localization    0.000000
dtype: float64


In [10]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
img_generator = ImageDataGenerator(
    rotation_range=180,
    horizontal_flip=True,
    vertical_flip=True, 
    rescale=1/255., 
    validation_split=0.3
    )

In [11]:
train_generator = img_generator.flow_from_dataframe(
    dataframe=data,
    directory="/Users/xxx/Documents/Jedha/final_project/Data/HAM10000_images",
    x_col="image_id", 
    y_col="dx",
    target_size=(128,128),
    class_mode = "sparse", 
    batch_size=200, 
    shuffle = True,
    seed=0,
    subset = "training"
)

val_generator = img_generator.flow_from_dataframe(
    dataframe=data,
    directory="/Users/xxx/Documents/Jedha/final_project/Data/HAM10000_images",
    x_col="image_id",
    y_col="dx",
    target_size=(128,128),
    class_mode = "sparse", 
    batch_size=200, 
    shuffle = True,
    seed=0,
    subset = "validation"
)

Found 7011 validated image filenames belonging to 7 classes.
Found 3004 validated image filenames belonging to 7 classes.


In [12]:
imgs, targets = next(iter(train_generator))

In [13]:
imgs.shape

(200, 128, 128, 3)

In [14]:
from tensorflow.keras.layers import Conv2D, MaxPool2D

modelconv = tf.keras.Sequential(
    [   
        Conv2D(
            filters = 32, 
            kernel_size = (3,3), 
            strides = 1, 
            padding = "same",
            activation = "relu", 
            input_shape = (imgs.shape[1], imgs.shape[2], imgs.shape[3])
        ), # the input shape (height, width, channels)
     MaxPool2D(
         pool_size=2, # the size of the pooling window
         strides=2
     ), # the movement of the pooling on the input
     Conv2D(
         filters = 64, 
         kernel_size = (3,3), 
         strides = 1, 
         padding = "same",
         activation = "relu"
     ),
     MaxPool2D(2,2),
     Conv2D(
         filters = 128, 
         kernel_size = (3,3), 
         strides = 1, 
         padding = "same",
         activation = "relu"
     ),
     MaxPool2D(2,2),
     Conv2D(
         filters = 256, 
         kernel_size = (3,3), 
         strides = 1, 
         padding = "same",
         activation = "relu"
     ),
     MaxPool2D(2,2),
     tf.keras.layers.Flatten(), # this layer turns multi-dimensional images into flat objects
     tf.keras.layers.Dense(len(pd.unique(data['dx'])),"softmax")
     
    ]
)

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2021-11-18 14:59:01.873708: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2021-11-18 14:59:01.874363: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [15]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy
from tensorflow.keras.optimizers import Adam

modelconv.compile(optimizer=Adam(),
              loss=SparseCategoricalCrossentropy(),
              metrics=[SparseCategoricalAccuracy()])

In [17]:
modelconv.fit(train_generator, epochs = 10, validation_data=val_generator)

2021-11-18 14:59:44.583110: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2021-11-18 14:59:44.584329: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2021-11-18 14:59:44.731282: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


Epoch 1/10
36/36 [==============================] - ETA: 0s - loss: 1.1393 - sparse_categorical_accuracy: 0.6689

2021-11-18 15:00:36.446468: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


36/36 [==============================] - 73s 2s/step - loss: 1.1393 - sparse_categorical_accuracy: 0.6689 - val_loss: 1.0623 - val_sparse_categorical_accuracy: 0.6698
Epoch 2/10
36/36 [==============================] - 70s 2s/step - loss: 1.0023 - sparse_categorical_accuracy: 0.6701 - val_loss: 0.9789 - val_sparse_categorical_accuracy: 0.6768
Epoch 3/10
36/36 [==============================] - 68s 2s/step - loss: 0.9349 - sparse_categorical_accuracy: 0.6695 - val_loss: 0.9510 - val_sparse_categorical_accuracy: 0.6887
Epoch 4/10
36/36 [==============================] - 68s 2s/step - loss: 0.9008 - sparse_categorical_accuracy: 0.6824 - val_loss: 0.8925 - val_sparse_categorical_accuracy: 0.6874
Epoch 5/10
36/36 [==============================] - 68s 2s/step - loss: 0.8849 - sparse_categorical_accuracy: 0.6868 - val_loss: 0.9087 - val_sparse_categorical_accuracy: 0.6947
Epoch 6/10
36/36 [==============================] - 70s 2s/step - loss: 0.8527 - sparse_categorical_accuracy: 0.6902 - va

In [18]:
tf.keras.models.save_model(modelconv,'skin_model.hdf5')